# Bayesian Model Averaging Regression

In this notebook we will use Bayes Factors and then Bayesian Model Averaging (BMA) to understand a linear regression problem.  The data is SAT scores for all states in the US, with information on school expenditures, student participation, student to teacher ratio, and teacher salary.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from statsmodels.regression.linear_model import OLS
from statsmodels.tools import add_constant
from itertools import combinations

Load the data, and check the head.  Our data is a the SAT scores by U.S. state along with variables for each state that might impact the average SAT score.

In [ ]:
df = pd.read_csv('/kaggle/input/sat-score-data-by-state/Guber1999data.csv')
df.head()

We start our analysis with a linear regression model has Total SAT score as a function of spending.

In [ ]:
X = df[["Spend"]]
y = df["SATT"]
regr = OLS(y, add_constant(X)).fit()
regr.summary()

Note that we get something very suprising here.  The coefficient for spending is -20.8922, with a p-score of 0.006.  This means that the variable is significant, and that higher spending corresponds to lower SAT scores, on average.  While we certainly won't make the improper conclusion that spending *causes* a decrease in SAT scores, this is still counter-intuitive.

Let's use the seaborn library and a pairplot to look at the data:

In [ ]:
# Seaborn visualization library
import seaborn as sns
# Create the default pairplot
sns.pairplot(df);

# Bayesian Information Criterion and Bayes Factors

We want to use the Bayesian Information Criterion to analyze models.  There are different information criterion that can be computed and used to estiamte liklihoods.  The different ICs use different approximations (using different simplifying assumptions and Taylor Series approximations). Assuming that the errors are normaly distributed, the formula for **Bayesian Information Criterion (BIC)** is:
\begin{equation}
    \text{BIC} = n\ln(\hat{\sigma^2_e}) + k \ln(n)
\end{equation}
where $n$ is the number of observations (data values), $k$ is the number of predictor variables in the regression model, and $\hat{\sigma^2_e}$ is the error variance, or Mean Squared Error (MSE),
\begin{equation}
    \hat{\sigma^2_e} = \text{MSE} = \frac{1}{n}\sum_{i=1}^n {(y_i-\hat{y_i})}^2.
\end{equation}
Note that the BIC formula has two terms.  The frist term, $n\ln(\hat{\sigma^2_e})$, measures the goodness-of-fit, since lower MSE gives a lower valuer for the first term.  The second term, $k \ln(n)$, is a penalty on the number of input varialbes in the model, and increases as the number of input valirables increases.

We also want to consider the likelihood of this model, and the likelihood is given by
\begin{equation}
    p(x|M)=e^{-\text{BIC}/2}.
\end{equation}
which, sometimes for ease of computation is combined with the above formulas to get a formula for likelihood in terms of MSE,
\begin{equation}
    p(x|M)=\text{MSE}^{-n/2}n^{-k/2}.
\end{equation}
Sometimes it is more intuitive to write the formula as
\begin{equation}
    p(x|M)=\frac{1}{\text{MSE}^{n/2}}\frac{1}{n^{k/2}}.
\end{equation}

If we have two models $M_1$ and $M_2$ with prior probabilities $p(M_1)$ and $p(M_2)$, the **Bayes Factor** is the ratio
\begin{equation}
   {BF}_{12} =\frac{p(x|M_1)p(M_1)}{p(x|M_2)p(M_2)},
\end{equation}
which is just the likelihood ratio if we have equal (uninformative) priors.

In [ ]:
likelihood1 = np.exp(-regr.bic/2)

Just as a verification, we can check that the this likelihood gives us a value that is close to the likelihood provided by the OLS software.  Because these computaitons are approximations to integrals, we expect the values to be close but not exactly equal.

In [ ]:
print(np.log(likelihood1))
print(regr.llf)

Now we create a model that considerd both the Spending and PrcntTake (percentage of students who take the exam) variables, and record the likelihood.

In [ ]:
X = df[["Spend", "PrcntTake"]]
y = df["SATT"]
regr = OLS(y, add_constant(X)).fit()
likelihood2 = np.exp(-regr.bic/2)
regr.summary()

Examining the summary, we see that this model has a higher F-statistic (106.7 in comparison to the 8.128 for the previous model) and a lower BIC (498.5 in comparison to 572.4 for the previous model).  Both of these suggest that this 2-element model is better than the previous 1-element model. (Recall that the F-statistic is useful for comparing two models onlt when the models are nested - one model is a subset of the other - which is the case here.)

This second model has a coefficient of 12.2865 for the spend variable, and still a p-score of 0.006, now suggesting that, for the same percent of students taking the exam, higher spending corresponds to higher SAT scores.  

The PrcntTake variable has a coefficient of -2.8509, so it appears that the PrcntTake variable is explaining the drop in SAT scores.  Schools in states that have higher spending also have higher percentages of students taking the exam, and thus are less 'selective' about which students take the exam.

We can use the Bayes' Factor to compare the likelihoods of these two models.

In [ ]:
print(likelihood2/likelihood1)

This Bayes factor is extremely large, suggesting that the 2-element model is much more likely than the 1-element model, so we should trust its conclusions more strongly.  But while giving one answer, this raises new questions:  
* Do we just take the value of 12.2865 for the coefficent of the spend variable, or should it be something else?
* How does the coefficient for Spend change as we add or remove other variables?  
* Among all the other possible models, what else might they tell us?

We will use Bayesian Model Averaging (BMA) to address these questions, but first we check the full model for comparison.

In [ ]:
X = df[["Spend", "StuTeaRat", "Salary", "PrcntTake"]]
y = df["SATT"]
regr = OLS(y, add_constant(X)).fit()
bic4 = regr.bic
likelihood4 = np.exp(-bic4/2)
regr.summary()

Observe that the coefficient for Spend is now around 4.5, significantly less than the 12.3 we saw in the 2-element model.  Which value is correct?  (And really what does 'correct' mean....?)

The F-statistic and BIC score are between the values for the first two models we looked at, suggesting this model is somewhere between the previous two in terms of being 'correct'.  Again, this raises more questions:
* How do we interpret this new model in light of the previous two?
* Is this new model good enough that we should somehow consider the values of the coefficients as well?
* and what about the $2^4=16$ total number of models that could be made?  Should we look at each individualy?

Fortunately, this is exactly what Baeysian Model Averaging is designed to handle!

# Bayesian Model Averaging
Here we define the class that will perform our BMA analysis.

For any model $M_i$ (each model is defined by the set of predictor varialbes being used in the model), Bayes theorem tells us that the probability for $M_i$ is
\begin{equation}
p(M_i|X,y)=\frac{p(X,y|M_i)p(M_i)}{p(X,y)}.
\end{equation}

Using our previous formulas, this becomes,
\begin{equation}
p(M_i|X,y)=\frac{e^{−\text{BIC}_i/2}p(M_i)}{\sum_k e^{−\text{BIC}_k/2}p(M_k)}.
\end{equation}

So far, we have just done Bayesian analysis to compute a posterior probability distribution on the parameters.  But now we can do more with the 'averaging' part of BMA.

The probability for any predictor variable is the sum of the probabilities for all models contiaining that predictor variable, and the expected value for the coefficient of the predictor variable is the average value of the coefficient over all models containing the variable, weighted by the probability of each model.  That is,
\begin{equation}
p(X_k) = \sum_{M_i \text{such that } X_k\in M_i} p(M_i|X,y),
\end{equation}
and
\begin{equation}
E[\beta_k] = \sum_{M_i \text{such that } X_k\in M_i} p(M_i|X,y)\times \beta_k^{(i)},
\end{equation}
where $\beta_k^{(i)}$ is the coefficient of $X_k$ in model $M_i$.

Here is code for a BMA class that will do the Bayeisan Model Averaging.  An excessively commented version of this code is provided at the end of the notebook, which includes commenting for the model averaging steps and code functionality.

In [ ]:
class BMA:
    
    def __init__(self, y, X, **kwargs):
        # Setup the basic variables.
        self.y = y
        self.X = X
        self.names = list(X.columns)
        self.nRows, self.nCols = np.shape(X)
        self.likelihoods = np.zeros(self.nCols)
        self.coefficients = np.zeros(self.nCols)
        self.probabilities = np.zeros(self.nCols)
        self.names = list(X.columns)
        # Check the max model size. (Max number of predictor variables to use in a model.)
        # This can be used to reduce the runtime but not doing an exhaustive sampling.
        if 'MaxVars' in kwargs.keys():
            self.MaxVars = kwargs['MaxVars']
        else:
            self.MaxVars = self.nCols  
        # Prepare the priors if they are provided.
        # The priors are provided for the individual regressor variables.
        # The prior for a model is the product of the priors on the variables in the model.
        if 'Priors' in kwargs.keys():
            if np.size(kwargs['Priors']) == self.nCols:
                self.Priors = kwargs['Priors']
            else:
                print("WARNING: Provided priors error.  Using equal priors instead.")
                print("The priors should be a numpy array of length equal tot he number of regressor variables.")
                self.Priors = np.ones(self.nCols)  
        else:
            self.Priors = np.ones(self.nCols)  
        
    def fit(self):
        # Perform the Bayesian Model Averaging
        
        # Initialize the sum of the likelihoods for all the models to zero.  
        # This will be the 'normalization' denominator in Bayes Theorem.
        likelighood_sum = 0
        
        # To facilitate iterating through all possible models, we start by iterating thorugh
        # the number of elements in the model.  
        for num_elements in range(1,self.MaxVars+1): 
            
            # Make a list of all index sets of models of this size.
            model_index_sets = list(combinations(list(range(self.nCols)), num_elements)) 
            
            # Iterate through all possible models of the given size.
            for model_index_set in model_index_sets:
                
                # Compute the linear regression for this given model. 
                model_X = self.X.iloc[:,list(model_index_set)]
                model_regr = OLS(self.y, model_X).fit()
                
                # Compute the likelihood (times the prior) for the model. 
                model_likelihood = np.exp(-model_regr.bic/2)*np.prod(self.Priors[list(model_index_set)])
                print("Model Variables:",model_index_set,"likelihood=",model_likelihood)
                # Add this likelihood to the running tally of likelihoods.
                likelighood_sum = likelighood_sum + model_likelihood
                
                # Add this likelihood (times the priors) to the runny tally
                # of likelihoods for each variable in the model.
                for idx, i in zip(model_index_set, range(num_elements)):
                    self.likelihoods[idx] = self.likelihoods[idx] + model_likelihood
                    self.coefficients[idx] = self.coefficients[idx] + model_regr.params[i]*model_likelihood

        # Divide by the denominator in Bayes theorem to normalize the probabilities 
        # sum to one.
        self.probabilities = self.likelihoods/likelighood_sum
        self.coefficients = self.coefficients/likelighood_sum
        
        # Return the new BMA object as an output.
        return self
        
    def summary(self):
        # Return the BMA results as a data frame for easy viewing.
        df = pd.DataFrame([self.names, list(self.probabilities), list(self.coefficients)], 
             ["Variable Name", "Probability", "Avg. Coefficient"]).T
        return df  

Now we split our data into input X dataframe and an output y dataframe, and run our BMA analysis.

In [ ]:
X = df[["Spend", "StuTeaRat", "Salary", "PrcntTake"]]
y = df["SATT"]
result = BMA(y,add_constant(X)).fit()
result.summary()

Now we can consider the full BMA analysis from the regression model.

First, we observe that the probability for the PrcntTake variable is 1.  This means that the models that did not include this where so unlikely that we probably shouldn't even consider them.

Second, pertaining to the origonal question of the relationship between spending and SAT scores, we see that, all other things being equal, increase in spending corresponds to an increase in SAT scores, and the expected value for the coefficient, after considering all possible models, is 8.47046.  This is not as high as what we saw in the model that considered just spending and percent of students taking the exam (about 12), buch much higher (and more reasonable) than the model that considered just spending alone (which had a coefficient of -2.85).  

Of course, there may be some direct or indirect relationships between spending and student-to-teacher ratio and teacher salary that should also be considered.  Taken as a whole, this single BMA output gives a much more complete picture than any individual models, or even consideration of two models.

Last, we should always consider that there are some other underlying variables that are not included in the analysis.  That is, what other data could have been collected about each state that might also be affecting the outcome SAT scores, and how would these variables affect the analysis.  From a model averaging perspective, we are considering whether our set of predictor varailbes is 'complete'.  The analysis only considers variables we put into it, and the results are only valid for these variables.

In [ ]:
# This is an excessively commented version of the code for Bayeisan Model Averaging.

class BMA:
    # After the class definition, we have a sequence of 'methods' associated with the class, 
    # which are just functions that are connected to the class and get internal access 
    # to the internal variables of the class.  The 'self' variable within the class always 
    # refers to class itself.  Varialbes with names of the form self.varname will be 
    # accesible in the methods within the class, and accesible outside the class as 
    # classname.varname.
    
    def __init__(self, y, X, **kwargs):
        # This __init__ function is the initilization method and runs when the class 
        # is created. Here we just use it to attached the X and y variables to the class, 
        # compute basic shape variables and attach them, and build some placeholder 
        # variables for our BMA analysis.  The variable names are self-explanatory.
        self.y = y
        self.X = X
        self.names = list(X.columns)
        self.nRows, self.nCols = np.shape(X)
        self.likelihoods = np.zeros(self.nCols)
        self.coefficients = np.zeros(self.nCols)
        self.probabilities = np.zeros(self.nCols)
        self.names = list(X.columns)
        # Check the max model size. (Max number of predictor variables to use in a model.)
        # This can be used to reduce the runtime but not doing an exhaustive sampling.
        if 'MaxVars' in kwargs.keys():
            self.MaxVars = kwargs['MaxVars']
        else:
            self.MaxVars = self.nCols  
        # Prepare the priors if they are provided.
        # The priors are provided for the individual regressor variables.
        # The prior for a model is the product of the priors on the variables in the model.
        if 'Priors' in kwargs.keys():
            if np.size(kwargs['Priors']) == self.nCols:
                self.Priors = kwargs['Priors']
            else:
                print("WARNING: Provided priors error.  Using equal priors instead.")
                print("The priors should be a numpy array of length equal tot he number of regressor variables.")
                self.Priors = np.ones(self.nCols)  
        else:
            self.Priors = np.ones(self.nCols)  
        
    def fit(self):
        # In this fit method, we are doing our model averaging.  This is a Baeysian 
        # process, where in general we consider different values of the parameters and 
        # use Bayes Theorem to compute a probability for each set of parameter values, 
        # resulting in a probability distribution for the parameters.
        
        # The parameters we want to estimate is whether or not to include each available 
        # predictor variable.  This means we want to assign a probability to the options 
        # {include, do not include} for each variable. This gives the probability of 
        # including the variable in the model.
        
        # From a Bayesian Statistics sampleing perspective, we are going to compute all 
        # possible parameter combinations as the default.  This is only feasible for 
        # problems with only a few predictor variables.  The keyward MaxVars can be used 
        # to reduce the number of models.
        
        # We initialize the sum of the likelihoods for all the models to zero.  
        # This will be the denominatory in Bayes Theorem, and we will apply it to 
        # normalize in the end.
        likelighood_sum = 0
        
        # To facilitate iterating through all possible models, we start by iterating thorugh
        # the number of elements in the model.  The number of elements is the number of 
        # predictor variables.
        for num_elements in range(1,self.MaxVars+1): 
            
            # Make a list of all index sets of models of this size.
            # For example, if there are 4 predictor variables, this will output the list
            # [(0,1), (0,2), (0,3), (1,2), (1,2), (2,3)].
            model_index_sets = list(combinations(list(range(self.nCols)), num_elements)) 
            # We now iterate through all possible models of the given size.
            for model_index_set in model_index_sets:
                
                # This is where the model averaging happens.
                # First, we compute the linear regression for this given model.  
                # (In other words, we select the set of input variables in model_index_set, 
                # and compute the linear regression model using just these variables.)  
                # We do this using OLS from the statsmodels package.  In our notation, 
                # henceforth any variable beginning with model_ is a variable that is 
                # just for this specific model.
                model_X = self.X.iloc[:,list(model_index_set)]
                model_regr = OLS(self.y, model_X).fit()
                
                # We compute the likelihood for the model from the BIC provided by OLS. 
                # This could alternatively be computed using AIC, the likelihood provided 
                # by OLS, or from RSS using the formula described previously.
                # NOTE:  This is actually the likelihood times the prior.
                model_likelihood = np.exp(-model_regr.bic/2)*np.prod(self.Priors[list(model_index_set)])
                print("Model Variables:",model_index_set,"likelihood=",model_likelihood)
                # Add this likelihood to the running tally of likelihoods for the denominator
                # in Bayes theorem.
                likelighood_sum = likelighood_sum + model_likelihood
                
                # The key step in model averaging for regression is that for each 
                # predictor variable, the probability that the variable should be included 
                # is the sum of all probabilities for all models that include the given 
                # variable.  This is equal to the sum of likelihoodsfor all models 
                # that include the given variable divided by the total likelihood.  
                
                # The other component of model average is that we can compute the 
                # average value of any any varaible over all the models, where this 
                # average is weighted by the probability of each model.  This gives the 
                # expected value for the variable given all the models being considered. 
                # In the following loop we compute the average value for the coefficients
                # for each variable.
                
                # following loop we iterate through all variables in the model 
                # (using their indexes), add the likelihood for the model to the vaiable 
                # tracking the likelihoods for each variable present in the model.  
                # We also add the coefficent for each variable (weighted by 
                # its likelihood) to the variable for the averaged coefficients.
                # [NOTE: idx is the index for the predictor variable within the set of 
                # all predictor variables and i is the index for this same 
                # predictor variable in the current regression model.]
                for idx, i in zip(model_index_set, range(num_elements)):
                    self.likelihoods[idx] = self.likelihoods[idx] + model_likelihood
                    self.coefficients[idx] = self.coefficients[idx] + model_regr.params[i]*model_likelihood

        # Now we divide by the denominator in Bayes theorem to normalize the probabilities to one.
        self.probabilities = self.likelihoods/likelighood_sum
        self.coefficients = self.coefficients/likelighood_sum
        
        # Having updated all the internal varaibles with our Bayeisan Model Averaing, 
        # we return the new BMA object as an output.
        return self
        
    def summary(self):
        # This function just takes the Bayesian Model Averaging analysis and returns
        # it as a data frame which makes it easier to veiw.
        df = pd.DataFrame([self.names, list(self.probabilities), list(self.coefficients)], 
             ["Variable Name", "Probability", "Avg. Coefficient"]).T
        return df        